In [25]:
import pandas as pd
import numpy as np

In [26]:
Parkhäuser = pd.read_csv("../Final_Data/Cleaned/1000014cleaned.csv")
Flugbewegungen = pd.read_csv("../Final_Data/Cleaned/Airport_traffic_hourly_cleaned.csv")
Gasverbrauch = pd.read_csv("../Final_Data/Cleaned/Gasverbrauch_cleaned.csv")
#Schallmessungen = pd.read_csv("../Final_Data/Cleaned/Schallmessungen_cleaned.csv", sep=";") -> zu viele Datenpunkte fehlen
Stromverbrauch = pd.read_csv("../Final_Data/Cleaned/Stromverbrauch_Basel_hourly_cleaned.csv")
Wetterdaten = pd.read_csv("../Final_Data/Cleaned/Wetterdaten_Basel_cleaned22_24.csv")
Verkehrsdaten = pd.read_csv("../Final_Data/Cleaned/1000006cleaned.csv")

In [27]:

Parkhäuser.rename(columns={"RoundedTime": "Datum"}, inplace=True)
Parkhäuser.head()
#Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.head()
#Gasverbrauch.head()
#Schallmessungen.drop(columns=["Unnamed: 0"], inplace=True)
#Schallmessungen.head()
#Stromverbrauch.drop(columns=["Unnamed: 0"], inplace=True)
#Stromverbrauch.head()
#Wetterdaten.head()
#Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"}, inplace=True)
#Verkehrsdaten.head()




,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [28]:
Parkhäuser.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,Claramattechange,Elisabethenchange,Steinenchange,Kunstmuseumchange,Messechange,Anfoschange,Centralbahnparkingchange
0,2022-01-01 00:00:00+00:00,-7.0,-6.0,-15.0,-19.0,0.0,-2.0,-2.0,-3.0,-1.0,-23.0,-5.0,-10.0,-4.0,-7.0,0.0,-2.0
1,2022-01-01 01:00:00+00:00,-7.0,-1.0,-4.0,-9.0,-1.0,0.0,0.0,0.0,0.0,0.0,-9.0,-19.0,0.0,-8.0,-1.0,-1.0
2,2022-01-01 02:00:00+00:00,-4.0,0.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,-12.0,-23.0,0.0,-8.0,0.0,0.0
3,2022-01-01 03:00:00+00:00,-3.0,0.0,-1.0,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,-15.0,-10.0,0.0,-2.0,0.0,0.0
4,2022-01-01 04:00:00+00:00,-2.0,0.0,2.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,-1.0,-7.0,0.0,-2.0,0.0,1.0


In [29]:
Flugbewegungen.head()



,Unnamed: 0,Datum,Hour,Traffic,Weekday
0,0,2022-01-01 00:00:00+00:00,0,1.0,Saturday
1,1,2022-01-01 00:00:00+00:00,1,0.0,Saturday
2,2,2022-01-01 00:00:00+00:00,2,0.0,Saturday
3,3,2022-01-01 00:00:00+00:00,3,0.0,Saturday
4,4,2022-01-01 00:00:00+00:00,4,0.0,Saturday


In [30]:
Verkehrsdaten = Verkehrsdaten.rename(columns={"DateTimeFrom": "Datum"})
print(Verkehrsdaten['Datum'].isna().sum())


0


In [31]:
#standardise the data
from sklearn.preprocessing import StandardScaler
import numpy as np
import pickle

dfs = [Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten]

# Dictionary to store the mean and standard deviation for each dataframe
scalers = {}

# Loop through each dataframe
for i, df in enumerate(dfs):
    numeric_cols = df.select_dtypes(include=[np.number])
    if not numeric_cols.empty:
        if i == 1:  # Exclude 'Hour' column for Flugbewegungen
            numeric_cols = numeric_cols.drop(columns=['Hour'])
        print(f"Processing df{i} with columns: {numeric_cols.columns}")
        scaler = StandardScaler()
        # Fit the scaler on the dataframe and transform the data
        scaled_data = scaler.fit_transform(numeric_cols)
        
        # Save the mean and standard deviation
        scalers[f'df{i}'] = {'mean': scaler.mean_, 'scale': scaler.scale_}
        
        # Replace the original data with the standardized data
        df[numeric_cols.columns] = scaled_data
        dfs[i] = df
    else:
        print(f"df{i} has no numeric columns or is empty.")

# Assign the transformed dataframes back to their original names
Parkhäuser, Flugbewegungen, Gasverbrauch, Stromverbrauch, Wetterdaten, Verkehrsdaten = dfs

# Save the scalers dictionary for later use
with open('scalers.pkl', 'wb') as f:
    pickle.dump(scalers, f)


Processing df0 with columns: Index(['Rebgassechange', 'Clarahuuschange', 'Citychange', 'Storchenchange',
       'Post Baselchange', 'Aeschenchange', 'Bahnhof Südchange',
       'Bad. Bahnhofchange', 'Europechange', 'Claramattechange',
       'Elisabethenchange', 'Steinenchange', 'Kunstmuseumchange',
       'Messechange', 'Anfoschange', 'Centralbahnparkingchange'],
      dtype='object')
Processing df1 with columns: Index(['Unnamed: 0', 'Traffic'], dtype='object')
Processing df2 with columns: Index(['Gasverbrauch'], dtype='object')
Processing df3 with columns: Index(['Stromverbrauch'], dtype='object')
Processing df4 with columns: Index(['Unnamed: 0', 'Basel Temperature [2 m elevation corrected]',
       'Basel Precipitation Total', 'Basel Wind Speed [10 m]',
       'Basel Wind Direction [10 m]'],
      dtype='object')
Processing df5 with columns: Index(['350n_sumPW', '350n_sumLief', '350n_sumLW', '350v_sumPW',
       '350v_sumLief', '350v_sumLW', '352n_sumPW', '352n_sumLief',
       '352

In [32]:
print(type(Verkehrsdaten))
Wetterdaten.drop(columns=["Unnamed: 0"], inplace=True)
Flugbewegungen.drop(columns=["Unnamed: 0"], inplace=True)
Gasverbrauch.sort_values(by=["Datum"], inplace=True)

<class 'pandas.core.frame.DataFrame'>


In [33]:
# Ensure 'Datum' is in datetime format
Flugbewegungen['Datum'] = pd.to_datetime(Flugbewegungen['Datum'])

# Add the 'Hour' column as timedelta
Flugbewegungen['Datum'] = Flugbewegungen['Datum'] + pd.to_timedelta(Flugbewegungen['Hour'], unit='h')

# Display the first 25 rows
Flugbewegungen.head(25)

Flugbewegungen["Datum"].dtype

datetime64[ns, UTC]

In [34]:
                                                                                                # The merge isnt correct because it erases the Hour format and therefore setting the entire Day to one value?!
# Merge all dataframes on the 'Datum' column
final_df = Parkhäuser.merge(Wetterdaten, on='Datum', how='inner') \
                      .merge(Stromverbrauch, on='Datum', how='inner') \
                      .merge(Verkehrsdaten, on='Datum', how='inner') \
                      .merge(Gasverbrauch, on='Datum', how='inner') \
#                      .merge(Flugbewegungen, on='Datum', how='inner') \

#                      .merge(Schallmessungen, on='Datum', how='inner') \ -> too many NaN values
# Display the first few rows of the merged dataframe
final_df.head(20)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.151270,...,-1.363950,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
1,2022-01-01 01:00:00+00:00,-0.489808,-0.193898,-0.043944,-0.525891,-0.191296,-0.000173,-0.000190,-0.000462,-0.000203,...,-1.363950,0.745474,-0.744920,-0.955306,-0.781539,-1.055084,-1.048762,-0.773270,-1.072471,0.357518
2,2022-01-01 02:00:00+00:00,-0.279837,-0.000213,-0.011259,-0.058515,-0.000054,-0.000173,-0.122840,-0.000462,-0.000203,...,-1.363950,0.575505,-0.744920,-0.937353,-0.571421,-1.055084,-1.057758,-0.416962,-1.072471,0.385414
3,2022-01-01 03:00:00+00:00,-0.209846,-0.000213,-0.011259,-0.350625,-0.000054,-0.000173,-0.000190,-0.000462,-0.000203,...,-1.363950,0.269563,-0.744920,-1.045068,-0.571421,-1.055084,-1.201702,-0.416962,-1.072471,0.617131
4,2022-01-01 04:00:00+00:00,-0.139856,-0.000213,0.021425,0.116751,-0.000054,-0.000173,0.245112,-0.000462,0.150864,...,-1.363950,0.575505,-0.331206,-1.063021,-0.781539,-1.055084,-1.255681,-0.654501,-1.072471,1.109860
5,2022-01-01 05:00:00+00:00,0.140106,-0.000213,0.925692,-0.233781,-0.000054,0.128787,-0.000190,0.052141,-0.000203,...,-1.363950,0.915442,-0.538063,-1.036092,-0.781539,-1.055084,-1.147723,-0.535731,-1.072471,1.670331
6,2022-01-01 06:00:00+00:00,-0.069865,0.193473,-0.022154,-0.116937,-0.000054,0.128787,-0.122840,-0.000462,-0.151270,...,-1.363950,0.201576,-0.538063,-1.161759,-0.571421,0.004582,-1.255681,-0.773270,-1.072471,1.455102
7,2022-01-01 07:00:00+00:00,0.000125,-0.193898,0.054109,-0.058515,-0.000054,-0.000173,0.122461,0.052141,-0.151270,...,-1.363950,0.337550,-0.331206,-1.089949,-0.781539,-1.055084,-1.210699,-0.773270,-1.072471,1.472270
8,2022-01-01 08:00:00+00:00,0.000125,0.193473,0.141268,-0.000093,-0.000054,-0.000173,0.613065,-0.053065,-0.151270,...,-1.363950,0.813461,-0.744920,-1.000187,-0.676480,-0.525251,-1.048762,-0.654501,-0.611608,1.555734
9,2022-01-01 09:00:00+00:00,-0.069865,1.161899,0.173952,-0.175359,0.573671,-0.000173,0.613065,0.104744,-0.151270,...,-1.356259,1.391353,-0.331206,-0.919401,-0.256244,-0.525251,-0.976790,-0.535731,-0.611608,1.401258


In [35]:
final_df.tail(25)

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
24566,2024-10-20 14:00:00+00:00,-1.399684,-0.000213,-0.621367,-1.752754,-0.000054,-0.387053,2.698128,-0.842113,0.301931,...,0.097193,-0.206348,-0.538063,0.633490,-0.781539,0.004582,0.408669,0.414423,-0.150745,-0.716121
24567,2024-10-20 15:00:00+00:00,0.070116,-0.000213,-0.752105,-1.285378,-0.000054,-0.773932,-3.066460,-0.421287,0.452998,...,0.089503,-0.002386,-0.538063,0.875848,0.374110,0.004582,0.723545,0.176884,-0.150745,-0.654515
24568,2024-10-20 16:00:00+00:00,-0.489808,-0.000213,-0.338103,-0.701157,-0.000054,-0.773932,-1.226698,-0.736906,-0.151270,...,-0.125824,-0.172354,-0.538063,0.660418,0.163992,0.004582,0.462647,-0.298193,-0.611608,-0.551452
24569,2024-10-20 17:00:00+00:00,-0.279837,-0.000213,-0.152892,-0.642735,-0.000054,-0.516012,-1.104047,-0.842113,0.150864,...,-0.148894,-0.274335,-0.951777,0.148772,0.058933,-0.525251,0.003827,-0.060654,-0.150745,-0.463764
24570,2024-10-20 18:00:00+00:00,-0.209846,-0.000213,-0.283629,-0.525891,-0.000054,-0.000173,-0.858745,-0.526494,0.301931,...,-0.610308,-0.376316,-0.951777,-0.362873,-0.571421,-1.055084,-0.104131,0.295653,-1.072471,-0.471266
24571,2024-10-20 19:00:00+00:00,-0.209846,-0.000213,-0.120207,-1.285378,-0.000054,-0.129133,0.245112,-0.368684,-0.000203,...,-0.717971,-0.206348,-0.951777,-0.703971,-0.466362,-1.055084,-0.410012,0.176884,-1.072471,-0.521984
24572,2024-10-20 20:00:00+00:00,-0.069865,-0.968639,0.032320,-1.051690,-0.000054,-0.129133,-0.981396,-0.158272,-0.000203,...,-0.910226,-0.342323,-0.951777,-0.892472,-0.676480,-1.055084,-0.949801,-0.654501,-1.072471,-0.666680
24573,2024-10-20 21:00:00+00:00,0.000125,-0.000213,-0.468840,-0.233781,-0.000054,-0.000173,-0.245491,-0.000462,-0.151270,...,-1.117862,-0.410310,-0.951777,-1.045068,-0.781539,-1.055084,-1.183709,-0.535731,-1.072471,-0.789995
24574,2024-10-20 22:00:00+00:00,0.000125,-0.000213,-0.283629,-0.000093,-0.000054,-0.000173,-0.000190,-0.053065,-0.000203,...,-1.233216,-0.376316,-0.951777,-1.116878,-0.466362,-1.055084,-1.282671,-0.654501,-0.611608,-0.824848
24575,2024-10-20 23:00:00+00:00,0.000125,-0.000213,-0.000365,-0.000093,-0.000054,-0.000173,0.122461,-0.000462,-0.000203,...,-1.225525,-0.410310,-0.951777,-1.233569,-0.781539,-1.055084,-1.363639,-0.773270,-1.072471,-0.833042


In [36]:
test_dataFrame = [Parkhäuser,
                Wetterdaten,
                Flugbewegungen,
                Gasverbrauch,
                Stromverbrauch,
                Verkehrsdaten]

for i in test_dataFrame:
    print(i['Datum'].max())

2024-10-21 14:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-22 23:00:00+00:00
2024-11-24 04:00:00+00:00
2024-11-20 22:00:00+00:00
2024-11-19 22:00:00+00:00


# Check for rows with NaN values
nan_rows = final_df[final_df.isnull().any(axis=1)]

# Display the rows with NaN values
nan_rows.head()

In [37]:
type(final_df[["Datum"]])

pandas.core.frame.DataFrame

In [38]:
final_df.to_csv("../Working_DataFrame.csv", index=False)


#final_df.tail(40)
final_df['Datum'].dtype
import numpy as np

cutoff = np.datetime64('2024-11-20 10:00:00')
final_df = final_df[final_df['Datum'] <= cutoff]
final_df.tail(40)

In [39]:
Flugbewegungen.head()

,Datum,Hour,Traffic,Weekday
0,2022-01-01 00:00:00+00:00,0,-1.142151,Saturday
1,2022-01-01 01:00:00+00:00,1,-1.295479,Saturday
2,2022-01-01 02:00:00+00:00,2,-1.295479,Saturday
3,2022-01-01 03:00:00+00:00,3,-1.295479,Saturday
4,2022-01-01 04:00:00+00:00,4,-1.295479,Saturday


In [40]:

Stromverbrauch["Datum"].dtype
Wetterdaten["Datum"].dtype
Gasverbrauch["Datum"].dtype

Flugbewegungen["Datum"] = Flugbewegungen["Datum"].astype(str)
Flugbewegungen["Datum"].dtype

dtype('O')

In [41]:
#DF NN
DF_NN = Gasverbrauch.merge(Stromverbrauch, on='Datum', how='left') \
                      .merge(Wetterdaten, on='Datum', how='left') \
                      .merge(Flugbewegungen, on='Datum', how='left')

In [42]:
DF_NN.head()

# Convert the date column to datetime format
DF_NN['Datum'] = pd.to_datetime(DF_NN['Datum'])

# Filter the dataframe to keep only dates before the specified date and time
DF_NN = DF_NN[DF_NN['Datum'] <= '2024-10-27 13:00']
DF_NN.head()


,Gasverbrauch,Datum,Stromverbrauch,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m],Hour,Traffic,Weekday
0,0.294494,2022-01-01 00:00:00+00:00,-1.116259,-0.926467,-0.314805,-0.694066,0.115891,0.0,-1.142151,Saturday
1,0.357518,2022-01-01 01:00:00+00:00,-1.368317,-1.041967,-0.314805,-0.918024,0.047361,1.0,-1.295479,Saturday
2,0.385414,2022-01-01 02:00:00+00:00,-1.523703,-1.162435,-0.314805,-0.857675,-0.143112,2.0,-1.295479,Saturday
3,0.617131,2022-01-01 03:00:00+00:00,-1.546518,-1.135112,-0.314805,-0.831118,-0.006355,3.0,-1.295479,Saturday
4,1.109860,2022-01-01 04:00:00+00:00,-1.432685,-1.163677,-0.314805,-0.687750,-0.148316,4.0,-1.295479,Saturday


Create a Dataframe only for the first year, to test if the computer can handle it 


In [43]:
DF_NN_Year = DF_NN.copy()
DF_NN_Year = DF_NN_Year[DF_NN_Year['Datum'] <= '2022-12-31 23:00']
DF_NN_Year.drop(columns=["Hour"], inplace=True)
DF_NN_Year.to_csv("./DF_NN_Year.csv", index=False)

In [44]:
Final_Target = pd.read_csv("../Final_Data/Cleaned/Final_Target_Data_Combined_resid_Trend.csv")
Final_Target = Final_Target[Final_Target['Datum'] <= '2022-12-31 23:00']
#Final_Target.drop(columns=["Datum"], inplace=True)
Final_Target.to_csv("./Final_Target_NN_Year.csv", index=False)

In [45]:

DF_NN.drop(columns=["Datum", "Hour"], inplace=True)

DF_NN.to_csv("./Working_DataFrame_NN.csv", index=False)

In [46]:
#2 NaN in Stromverbrauch 2022-10-30 01:00:00 & 2023-10-29 01:00:00

# Verkehrsdatenset hat 5 NaN values 2024-10-20 22:00:00 & 2024-10-20 23:00:00 & 2024-10-21 00:00:00 & 2024-10-21 01:00:00 & 2024-10-21 02:00:00	

# Wetter hat 5 NaN values 2023-12-22 00:00:00 & 2023-12-22 01:00:00 & 2023-12-22 02:00:00 & 2023-12-22 03:00:00 & 2023-12-22 04:00:00
# habe nochmal nachgeschaut und es ist der ganze 22 und 23 Dezember 2023 || ist aber nur eine Auflösung die ich ausgewählt habe, sollte mit der anderen Auflösung nicht mehr vorkommen


In [47]:
final_df.head()

,Datum,Rebgassechange,Clarahuuschange,Citychange,Storchenchange,Post Baselchange,Aeschenchange,Bahnhof Südchange,Bad. Bahnhofchange,Europechange,...,84111104v_sumPW,84111104v_sumLief,84111104v_sumLW,84111108n_sumPW,84111108n_sumLief,84111108n_sumLW,84111108v_sumPW,84111108v_sumLief,84111108v_sumLW,Gasverbrauch
0,2022-01-01 00:00:00+00:00,-0.489808,-1.162325,-0.163786,-1.110112,-0.000054,-0.258093,-0.245491,-0.158272,-0.151270,...,-1.36395,1.425346,-0.124348,-0.686018,-0.571421,-1.055084,-0.814853,-0.416962,-1.072471,0.294494
1,2022-01-01 01:00:00+00:00,-0.489808,-0.193898,-0.043944,-0.525891,-0.191296,-0.000173,-0.000190,-0.000462,-0.000203,...,-1.36395,0.745474,-0.744920,-0.955306,-0.781539,-1.055084,-1.048762,-0.773270,-1.072471,0.357518
2,2022-01-01 02:00:00+00:00,-0.279837,-0.000213,-0.011259,-0.058515,-0.000054,-0.000173,-0.122840,-0.000462,-0.000203,...,-1.36395,0.575505,-0.744920,-0.937353,-0.571421,-1.055084,-1.057758,-0.416962,-1.072471,0.385414
3,2022-01-01 03:00:00+00:00,-0.209846,-0.000213,-0.011259,-0.350625,-0.000054,-0.000173,-0.000190,-0.000462,-0.000203,...,-1.36395,0.269563,-0.744920,-1.045068,-0.571421,-1.055084,-1.201702,-0.416962,-1.072471,0.617131
4,2022-01-01 04:00:00+00:00,-0.139856,-0.000213,0.021425,0.116751,-0.000054,-0.000173,0.245112,-0.000462,0.150864,...,-1.36395,0.575505,-0.331206,-1.063021,-0.781539,-1.055084,-1.255681,-0.654501,-1.072471,1.109860


In [48]:
import pandas as pd
df = final_df.copy()
# Ensure 'Datum' is a datetime object
df['Datum'] = pd.to_datetime(df['Datum'])

# Create a complete range of hourly timestamps from the first to the last point
full_range = pd.date_range(start=df['Datum'].min(), end=df['Datum'].max(), freq='h')

# Identify missing timestamps
missing_timestamps = full_range.difference(df['Datum'])

# Display the results
if missing_timestamps.empty:
    print("No missing hourly timestamps. All hours are accounted for.")
else:
    print("Missing hourly timestamps:")
    print(missing_timestamps)

No missing hourly timestamps. All hours are accounted for.
